In [2]:

!pip install -U layoutparser
!pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2'
!pip install layoutparser[ocr]


  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.4) to /tmp/pip-install-nd_0sxue/detectron2_70698413d20148679fa2a6a02d707be3
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-nd_0sxue/detectron2_70698413d20148679fa2a6a02d707be3
  Running command git checkout -q 4aca4bdaa9ad48b8e91d7520e0d0815bb8ca0fb1
  Resolved https://github.com/facebookresearch/detectron2.git to commit 4aca4bdaa9ad48b8e91d7520e0d0815bb8ca0fb1
  Preparing metadata (setup.py) ... done


In [10]:
!git clone https://github.com/Layout-Parser/layout-parser.git


fatal: destination path 'layout-parser' already exists and is not an empty directory.


In [7]:
import layoutparser as lp
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import cv2
import fitz
import json
from PIL import Image
!pip install pillow==9.5.0

In [29]:

model = lp.Detectron2LayoutModel(
    'lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
    label_map={0: "List", 1: "Table", 2: "Figure"}
)


In [31]:
import os
import json
import fitz  # PyMuPDF
import layoutparser as lp

def process_pdfs_in_directory(input_dir, output_dir):
    # Check if Detectron2 is available
    if not lp.is_detectron2_available():
        print("Detectron2 is not available. Please ensure it's installed properly.")
        return

    # Define layout model
    model = lp.Detectron2LayoutModel(
        'lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
        label_map={0: "List", 1: "Table", 2: "Figure"}
    )

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over all PDF files in the input directory
    for filename in os.listdir(input_dir):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(input_dir, filename)
            output_json_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.json")
            annotated_pdf_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_annotated.pdf")

            # Load PDF
            pdf_document = fitz.open(pdf_path)
            results = []

            # Process each page in the PDF
            for page_num in range(len(pdf_document)):
                page = pdf_document[page_num]

                # Convert PDF page to an image for layout detection
                pix = page.get_pixmap()
                image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

                # Run layout detection
                layout = model.detect(image)
                layout_data = layout.to_dict()

                # Append the layout data for the current page
                results.append({
                    "page": page_num + 1,
                    "layout": layout_data
                })

                # Annotate the PDF page with bounding boxes
                for block in layout:
                    x_1, y_1, x_2, y_2 = block.coordinates
                    rect = fitz.Rect(x_1, y_1, x_2, y_2)

                    # Set RGB color without alpha
                    rgb_color = (1, 0, 0)  # Red color for bounding box
                    page.draw_rect(rect, color=rgb_color, width=2)

            # Save the annotated PDF
            pdf_document.save(annotated_pdf_path)

            # Save results to JSON
            with open(output_json_path, "w") as json_file:
                json.dump(results, json_file, indent=4)

            print(f"Processed {filename}, JSON saved to {output_json_path}, annotated PDF saved to {annotated_pdf_path}")

# Example usage
process_pdfs_in_directory("/content/drive/MyDrive/Research/Police Records/Dataset/saved_pdfs", "/content/drive/MyDrive/Research/Police Records/Baselines/detectron2_output")


Processed 1715883096213-sio-1.pdf, JSON saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_output/1715883096213-sio-1.json, annotated PDF saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_output/1715883096213-sio-1_annotated.pdf
Processed 1715883858842-fjt-1.pdf, JSON saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_output/1715883858842-fjt-1.json, annotated PDF saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_output/1715883858842-fjt-1_annotated.pdf
Processed 1715882749531-bdd-35.pdf, JSON saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_output/1715882749531-bdd-35.json, annotated PDF saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_output/1715882749531-bdd-35_annotated.pdf
Processed 1715886035061-jjp-2.pdf, JSON saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_output/1715886035061-jjp-2.json, a

In [32]:
model = lp.Detectron2LayoutModel(
    'lp://TableBank/faster_rcnn_R_50_FPN_3x/config',
    label_map={0: "Table"}
)


In [33]:
import os
import json
import fitz  # PyMuPDF
import layoutparser as lp

def process_pdfs_in_directory(input_dir, output_dir):
    # Check if Detectron2 is available
    if not lp.is_detectron2_available():
        print("Detectron2 is not available. Please ensure it's installed properly.")
        return

    # Define layout model
    model = lp.Detectron2LayoutModel(
        'lp://TableBank/faster_rcnn_R_50_FPN_3x/config',
        label_map={0: "Table"}
    )

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over all PDF files in the input directory
    for filename in os.listdir(input_dir):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(input_dir, filename)
            output_json_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.json")
            annotated_pdf_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_annotated.pdf")

            # Load PDF
            pdf_document = fitz.open(pdf_path)
            results = []

            # Process each page in the PDF
            for page_num in range(len(pdf_document)):
                page = pdf_document[page_num]

                # Convert PDF page to an image for layout detection
                pix = page.get_pixmap()
                image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

                # Run layout detection
                layout = model.detect(image)
                layout_data = layout.to_dict()

                # Append the layout data for the current page
                results.append({
                    "page": page_num + 1,
                    "layout": layout_data
                })

                # Annotate the PDF page with bounding boxes
                for block in layout:
                    x_1, y_1, x_2, y_2 = block.coordinates
                    rect = fitz.Rect(x_1, y_1, x_2, y_2)

                    # Set RGB color without alpha
                    rgb_color = (1, 0, 0)  # Red color for bounding box
                    page.draw_rect(rect, color=rgb_color, width=2)

            # Save the annotated PDF
            pdf_document.save(annotated_pdf_path)

            # Save results to JSON
            with open(output_json_path, "w") as json_file:
                json.dump(results, json_file, indent=4)

            print(f"Processed {filename}, JSON saved to {output_json_path}, annotated PDF saved to {annotated_pdf_path}")

# Example usage
process_pdfs_in_directory("/content/drive/MyDrive/Research/Police Records/Dataset/saved_pdfs", "/content/drive/MyDrive/Research/Police Records/Baselines/detectron2_tablebank")


Processed 1715883096213-sio-1.pdf, JSON saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_tablebank/1715883096213-sio-1.json, annotated PDF saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_tablebank/1715883096213-sio-1_annotated.pdf
Processed 1715883858842-fjt-1.pdf, JSON saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_tablebank/1715883858842-fjt-1.json, annotated PDF saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_tablebank/1715883858842-fjt-1_annotated.pdf
Processed 1715882749531-bdd-35.pdf, JSON saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_tablebank/1715882749531-bdd-35.json, annotated PDF saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_tablebank/1715882749531-bdd-35_annotated.pdf
Processed 1715886035061-jjp-2.pdf, JSON saved to /content/drive/MyDrive/Research/Police Records/Baselines/detectron2_tablebank/171588